In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time
import jsonlines
import ast
import codecs
import langid

In [2]:
#ajar article from 1/3/2018 - todays date 8/7/2023
start_date = '2018/03/01'
today = datetime.now()
article_dates = pd.date_range(start_date,today).strftime("%Y/%m/%d")

In [3]:
hrefs = []

for article_date in tqdm(article_dates):
    while True:
        try:
            r = requests.get(f"https://ajar.com.my/{article_date}/")
            if r.status_code == 200 or r.status_code == 404:
                break
        except Exception as e:
            print(e)
            time.sleep(1.3)

    soup = BeautifulSoup(r.text, "lxml")
    div = soup.find_all('h2', attrs = {"class":"cs-entry__title"})

    if len(div) == 0:
        continue
        
    for i in range(len(div)):
        for j in range(len(div[i].find_all("a"))):
            hrefs.append(div[i].find_all("a")[j].get("href"))

100%|██████████████████████████████████████████████████████████████████████████████| 1956/1956 [22:35<00:00,  1.44it/s]


In [4]:
len(hrefs)

642

In [5]:
len(set(hrefs))

642

In [7]:
for href in tqdm(hrefs):
    while True:
        try:
            r = requests.get(href)
            if r.status_code == 200:
                break
        except Exception as e:
            print(e)
            time.sleep(3.9)
            
    soup = BeautifulSoup(r.text, "lxml")
    title = soup.find("h1", attrs = {"class":"cs-entry__title cs-entry__title-line"}).text.strip()
    
    div = soup.find_all('div', attrs = {"class":"entry-content"})
    p = [p.text for p in div[0].find_all(["p","h2"]) if len(p.text.split()) > 0]
    
    data = {'url': href, 'headline': title, 'content': p}
    
    with open('ajar-before.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

100%|████████████████████████████████████████████████████████████████████████████████| 642/642 [11:26<00:00,  1.07s/it]


In [8]:
# Specify the input and output file paths
input_file = "ajar-before.jsonl"
output_file = "ajar.jsonl"

langid.set_languages(['ms', 'en'])  #ISO 639-1 codes

# Open the input and output files
with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
    # Iterate over each line in the input file
    for line in reader:
        # Check if the content field is empty ([] or {})
        if line['content']:
            # Write the non-empty line to the output file
            decoded_list = []
            for string in line['content']:
                try:
                    decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
                except ValueError:
                    decoded_string = string
                decoded_list.append(decoded_string)
            line['content'] = decoded_list

            writer.write(line)